In [1]:
"""
A very simple bug-fixing agent in pure Python.

Idea:
- We have a broken function `add(a, b)` that incorrectly returns a - b.
- We have tests that define the correct behavior.
- The BugFixAgent tries a small set of code patches until the tests pass.
"""

import textwrap

In [2]:
# -----------------------------
# 1. The initial broken program
# -----------------------------
BROKEN_CODE = textwrap.dedent("""
def add(a, b):
    # BUG: This should be a + b, but it's written as a - b.
    return a - b
""")


# -----------------------------
# 2. The "environment" with tests
# -----------------------------
class BugFixEnvironment:
    def __init__(self, code: str):
        self.code = code  # current version of the code under test

    def run_tests(self):
        """
        Execute the current code and run tests on it.
        Returns (passed: bool, message: str)
        """
        namespace = {}

        try:
            # Execute the current code string so that `add` is defined in namespace
            exec(self.code, namespace, namespace)
        except Exception as e:
            return False, f"Code execution error: {e}"

        if "add" not in namespace:
            return False, "Function 'add' is not defined."

        add = namespace["add"]

        # Simple test cases
        tests = [
            ((1, 2), 3),
            ((-1, 5), 4),
            ((10, 0), 10),
        ]

        try:
            for (args, expected) in tests:
                result = add(*args)
                if result != expected:
                    return False, f"Test failed for add{args}: expected {expected}, got {result}"
        except Exception as e:
            return False, f"Runtime error while testing: {e}"

        return True, "All tests passed!"


In [3]:
# -----------------------------
# 3. The bug-fixing agent
# -----------------------------
class BugFixAgent:
    def __init__(self, env: BugFixEnvironment):
        self.env = env
        self.patches_tried = []

    def perceive(self):
        """Observe the outcome of running the tests."""
        passed, message = self.env.run_tests()
        return {"passed": passed, "message": message}

    def candidate_patches(self):
        """
        A small library of possible patches.
        In real systems, this could be huge; here we hard-code a few simple ones.
        Each patch is a function that takes the current code string and returns a new code string.
        """
        def replace_minus_with_plus(code: str) -> str:
            # naive: replace 'a - b' with 'a + b'
            return code.replace("a - b", "a + b")

        def replace_return_minus_with_plus_general(code: str) -> str:
            # slightly more general: replace 'return a - b' with 'return a + b'
            return code.replace("return a - b", "return a + b")

        # You can add more strategies here, in order.
        return [
            ("replace_minus_with_plus", replace_minus_with_plus),
            ("replace_return_minus_with_plus_general", replace_return_minus_with_plus_general),
        ]

    def decide_and_act(self, perception):
        """
        Based on the test result, decide what to do next.
        - If tests passed: stop.
        - If tests failed: choose the next untried patch, apply it to the code.
        """
        if perception["passed"]:
            print("Agent: Tests already pass, no fix needed.")
            return False  # no further action

        print(f"Agent: Tests failed with message: {perception['message']}")

        # Try patches one by one
        for name, patch_fn in self.candidate_patches():
            if name in self.patches_tried:
                continue  # already tried this patch

            print(f"Agent: Trying patch '{name}'...")
            new_code = patch_fn(self.env.code)

            # Mark this patch as used
            self.patches_tried.append(name)

            # Update environment's code
            self.env.code = new_code
            print("Agent: Patch applied. New code is:\n", self.env.code)
            return True  # action taken (patched)

        print("Agent: No more patches to try.")
        return False  # no action possible

    def run(self, max_iterations=5):
        """
        Run the perceive–decide–act loop for a limited number of iterations.
        """
        for step in range(1, max_iterations + 1):
            print(f"\n=== Agent Step {step} ===")
            perception = self.perceive()
            print("Environment test result:", perception["message"])

            if perception["passed"]:
                print("Agent: Goal achieved! Code passes all tests.")
                break

            action_taken = self.decide_and_act(perception)
            if not action_taken:
                print("Agent: Stopping. No further actions possible.")
                break


In [4]:
# -----------------------------
# 4. Run a demo
# -----------------------------
if __name__ == "__main__":
    print("Initial broken code:\n", BROKEN_CODE)

    env = BugFixEnvironment(BROKEN_CODE)
    agent = BugFixAgent(env)

    agent.run()

    print("\nFinal code after agent run:\n", env.code)


Initial broken code:
 
def add(a, b):
    # BUG: This should be a + b, but it's written as a - b.
    return a - b


=== Agent Step 1 ===
Environment test result: Test failed for add(1, 2): expected 3, got -1
Agent: Tests failed with message: Test failed for add(1, 2): expected 3, got -1
Agent: Trying patch 'replace_minus_with_plus'...
Agent: Patch applied. New code is:
 
def add(a, b):
    # BUG: This should be a + b, but it's written as a + b.
    return a + b


=== Agent Step 2 ===
Environment test result: All tests passed!
Agent: Goal achieved! Code passes all tests.

Final code after agent run:
 
def add(a, b):
    # BUG: This should be a + b, but it's written as a + b.
    return a + b

